# 🎯 XGBoost: Detección de Fraude

**Materia:** Ciencia de Datos  
**Autores:** Mariana Pérez Pérez, Yoriel Navier Carvajalino Vidal, Adriana Lucia Castro Carreño

---

## 📋 Objetivos de esta demostración:

1. ✅ Aplicar XGBoost a un problema real de clasificación
2. 🔍 **Demostrar cómo XGBoost selecciona variables** (Gain, Weight, Cover)
3. 📊 **Visualizar el proceso iterativo de aprendizaje**
4. ⚖️ **Mostrar el efecto de la regularización** (L1 y L2)
5. 🎨 **Ilustrar splits y thresholds** en la toma de decisiones
6. 📈 Evaluar el rendimiento con métricas apropiadas


## 📦 Paso 1: Configuración del Entorno

Instalamos e importamos todas las librerías necesarias.

In [ ]:
# Instalación de librerías
!pip install -q gdown xgboost

# Importaciones
import pandas as pd
import numpy as np
import gdown
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    precision_recall_curve, 
    auc,
    roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuraciones
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Librerías importadas correctamente")
print(f"📌 Versión de XGBoost: {xgb.__version__}")

## 📊 Paso 2: Carga y Análisis Exploratorio de Datos

Cargamos el dataset de detección de fraude en tarjetas de crédito.

In [ ]:
# Descarga automática del dataset
output_filename = "creditcard.csv"

if not os.path.exists(output_filename):
    print(f"📥 Descargando dataset...")
    file_id = "1JXhUJjoGnRBR6tUkvvPv8DFisZZI-Iwc"
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, output_filename, quiet=False)
else:
    print(f"✅ Dataset ya existe")

# Cargar datos
df = pd.read_csv(output_filename)
print(f"\n📐 Dimensiones: {df.shape}")
print(f"📊 Columnas: {df.shape[1]}")
print(f"📝 Registros: {df.shape[0]:,}")

# Mostrar primeras filas
display(df.head())

# Información del dataset
print("\n📋 Información del dataset:")
print(df.info())

### 🔍 Análisis del Desbalance de Clases

Este es un dataset **altamente desbalanceado**, característica común en problemas de detección de fraude.

In [ ]:
# Análisis del desbalance
class_counts = df['Class'].value_counts()
class_percentages = df['Class'].value_counts(normalize=True) * 100

print("🎯 Distribución de clases:")
print(f"   No Fraude (0): {class_counts[0]:,} ({class_percentages[0]:.2f}%)")
print(f"   Fraude (1):    {class_counts[1]:,} ({class_percentages[1]:.3f}%)")
print(f"\n⚖️ Ratio de desbalance: {class_counts[0]/class_counts[1]:.1f}:1")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
class_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribución de Clases (Cantidad)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Clase')
axes[0].set_ylabel('Cantidad')
axes[0].set_xticklabels(['No Fraude', 'Fraude'], rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Gráfico de pie
colors = ['#2ecc71', '#e74c3c']
axes[1].pie(class_counts, labels=['No Fraude', 'Fraude'], autopct='%1.3f%%', 
            colors=colors, startangle=90, textprops={'fontsize': 12})
axes[1].set_title('Proporción de Clases', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Implicación: Necesitamos usar scale_pos_weight para manejar el desbalance")

## 🔧 Paso 3: Preprocesamiento de Datos

Preparamos los datos para el modelado.

In [ ]:
# Separar características y variable objetivo
X = df.drop('Class', axis=1)
y = df['Class']

# Estandarización de Amount y Time
scaler = StandardScaler()
X['Amount'] = scaler.fit_transform(X[['Amount']])
X['Time'] = scaler.fit_transform(X[['Time']])

print("✅ Datos estandarizados")
print(f"📊 Features: {X.shape[1]}")
print(f"🎯 Samples: {X.shape[0]:,}")

# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n📚 Train set: {X_train.shape[0]:,} muestras")
print(f"🧪 Test set:  {X_test.shape[0]:,} muestras")

## 🚀 Paso 4: Entrenamiento de Modelos XGBoost

### 🎯 4.1: Modelo Base (Con Regularización)

Entrenamos un modelo con regularización para demostrar los conceptos del PDF.

In [ ]:
# Calcular scale_pos_weight para el desbalance
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"⚖️ scale_pos_weight calculado: {scale_pos_weight:.2f}")

# Modelo con regularización (como explica el PDF)
model_regularized = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    # 🔥 Regularización (L1 y L2 como en el PDF)
    reg_alpha=0.1,   # L1 regularization (Lasso)
    reg_lambda=1.0,  # L2 regularization (Ridge)
    subsample=0.8,
    colsample_bytree=0.8
)

# Entrenamiento con seguimiento del error
print("\n🎯 Entrenando modelo con regularización...")
model_regularized.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

print("✅ Modelo entrenado exitosamente")

### 📊 4.2: Visualización del Proceso Iterativo de Aprendizaje

**Como explica el PDF:** XGBoost aprende iterativamente, reduciendo el error en cada paso.

In [ ]:
# Obtener resultados de evaluación por iteración
results = model_regularized.evals_result()

# Crear visualización
fig, ax = plt.subplots(figsize=(12, 6))

# Plot para train y test
epochs = range(len(results['validation_0']['logloss']))
ax.plot(epochs, results['validation_0']['logloss'], 
        label='Train Error', linewidth=2, color='#3498db')
ax.plot(epochs, results['validation_1']['logloss'], 
        label='Test Error', linewidth=2, color='#e74c3c')

ax.set_xlabel('Número de Árboles (Iteraciones)', fontsize=12)
ax.set_ylabel('Log Loss (Error)', fontsize=12)
ax.set_title('📉 Proceso Iterativo de Aprendizaje de XGBoost\n(Cada árbol corrige errores del anterior)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Anotar puntos clave
min_idx = np.argmin(results['validation_1']['logloss'])
ax.annotate(f'Mejor modelo\n(Árbol {min_idx})', 
            xy=(min_idx, results['validation_1']['logloss'][min_idx]),
            xytext=(min_idx+10, results['validation_1']['logloss'][min_idx]+0.01),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=10, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n💡 Observación: El error disminuye con cada iteración")
print(f"   Mejor modelo en el árbol #{min_idx}")
print(f"   Error final (test): {results['validation_1']['logloss'][-1]:.6f}")

### 🔍 4.3: Comparación - Efecto de la Regularización

**Como explica el PDF:** La regularización (L1 y L2) previene el sobreajuste.

In [ ]:
# Modelo SIN regularización para comparar
print("🎯 Entrenando modelo SIN regularización (para comparar)...")
model_no_reg = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    # ❌ SIN regularización
    reg_alpha=0,
    reg_lambda=0,
    subsample=0.8,
    colsample_bytree=0.8
)

model_no_reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

print("✅ Ambos modelos listos para comparar")

# Comparación visual
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Modelo CON regularización
results_reg = model_regularized.evals_result()
epochs = range(len(results_reg['validation_0']['logloss']))
axes[0].plot(epochs, results_reg['validation_0']['logloss'], 
             label='Train', linewidth=2, color='#3498db')
axes[0].plot(epochs, results_reg['validation_1']['logloss'], 
             label='Test', linewidth=2, color='#e74c3c')
axes[0].set_xlabel('Iteraciones', fontsize=11)
axes[0].set_ylabel('Error (Log Loss)', fontsize=11)
axes[0].set_title('✅ CON Regularización (L1 + L2)\nModelo más robusto', 
                  fontsize=12, fontweight='bold', color='green')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Modelo SIN regularización
results_no_reg = model_no_reg.evals_result()
axes[1].plot(epochs, results_no_reg['validation_0']['logloss'], 
             label='Train', linewidth=2, color='#3498db')
axes[1].plot(epochs, results_no_reg['validation_1']['logloss'], 
             label='Test', linewidth=2, color='#e74c3c')
axes[1].set_xlabel('Iteraciones', fontsize=11)
axes[1].set_ylabel('Error (Log Loss)', fontsize=11)
axes[1].set_title('⚠️ SIN Regularización\nPosible sobreajuste', 
                  fontsize=12, fontweight='bold', color='red')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Análisis de overfitting
train_error_reg = results_reg['validation_0']['logloss'][-1]
test_error_reg = results_reg['validation_1']['logloss'][-1]
gap_reg = abs(train_error_reg - test_error_reg)

train_error_no_reg = results_no_reg['validation_0']['logloss'][-1]
test_error_no_reg = results_no_reg['validation_1']['logloss'][-1]
gap_no_reg = abs(train_error_no_reg - test_error_no_reg)

print("\n📊 Análisis de Sobreajuste (Overfitting):")
print("="*60)
print(f"{'Modelo':<25} {'Train Error':<15} {'Test Error':<15} {'Gap':<10}")
print("="*60)
print(f"{'CON Regularización':<25} {train_error_reg:<15.6f} {test_error_reg:<15.6f} {gap_reg:<10.6f}")
print(f"{'SIN Regularización':<25} {train_error_no_reg:<15.6f} {test_error_no_reg:<15.6f} {gap_no_reg:<10.6f}")
print("="*60)
print(f"\n💡 Gap menor = Mejor generalización")
print(f"   ✅ Modelo con regularización: {gap_reg:.6f}")
print(f"   ⚠️  Modelo sin regularización: {gap_no_reg:.6f}")

## 🎯 Paso 5: Selección y Evaluación de Variables

### 📊 5.1: Feature Importance - Las 3 Métricas de XGBoost

**Como explica el PDF**, XGBoost usa 3 métricas para evaluar variables:
- **Gain (Ganancia):** Cuánto mejora el modelo al usar esa variable
- **Weight (Peso):** Número de veces que aparece en las divisiones
- **Cover (Cobertura):** Cantidad de muestras afectadas

In [ ]:
# Crear figura con 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 1. GAIN (Ganancia) - La más importante
xgb.plot_importance(model_regularized, importance_type='gain', 
                   ax=axes[0], max_num_features=10, 
                   title='', color='#3498db')
axes[0].set_title('📈 GAIN (Ganancia)\nCuánto mejora el modelo', 
                 fontsize=12, fontweight='bold')
axes[0].set_xlabel('Gain Score', fontsize=10)

# 2. WEIGHT (Peso)
xgb.plot_importance(model_regularized, importance_type='weight', 
                   ax=axes[1], max_num_features=10,
                   title='', color='#e74c3c')
axes[1].set_title('⚖️ WEIGHT (Peso)\nFrecuencia de uso', 
                 fontsize=12, fontweight='bold')
axes[1].set_xlabel('Weight Score', fontsize=10)

# 3. COVER (Cobertura)
xgb.plot_importance(model_regularized, importance_type='cover', 
                   ax=axes[2], max_num_features=10,
                   title='', color='#2ecc71')
axes[2].set_title('📊 COVER (Cobertura)\nMuestras afectadas', 
                 fontsize=12, fontweight='bold')
axes[2].set_xlabel('Cover Score', fontsize=10)

plt.suptitle('🔍 Importancia de Variables en XGBoost - Las 3 Métricas', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Obtener las variables más importantes por GAIN
importance_dict = model_regularized.get_booster().get_score(importance_type='gain')
importance_sorted = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

print("\n🏆 TOP 10 Variables Más Importantes (por GAIN):")
print("="*50)
for i, (feature, score) in enumerate(importance_sorted[:10], 1):
    print(f"{i:2d}. {feature:<8} → Gain: {score:>10.2f}")
print("="*50)
print("\n💡 Estas variables tienen el mayor impacto en las predicciones")

### 🌳 5.2: Visualización de un Árbol - Splits y Thresholds

**Como explica el PDF:** XGBoost decide mediante splits (divisiones) y thresholds (umbrales).

In [ ]:
# Visualizar el primer árbol del modelo
fig, ax = plt.subplots(figsize=(20, 10))
xgb.plot_tree(model_regularized, num_trees=0, ax=ax)
plt.title('🌳 Visualización de un Árbol de Decisión en XGBoost\nCada nodo muestra: [Variable < Threshold]', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 En cada nodo del árbol:")
print("   - Se selecciona una variable (ej: 'V14')")
print("   - Se define un threshold (ej: < -1.5)")
print("   - Se divide la data según esa condición")
print("   - XGBoost elige el split con mayor GAIN")

## 📈 Paso 6: Evaluación del Modelo

### 🎯 6.1: Matriz de Confusión

In [ ]:
# Predicciones
y_pred = model_regularized.predict(X_test)
y_pred_proba = model_regularized.predict_proba(X_test)[:, 1]

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Visualización mejorada
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Fraude', 'Fraude'],
            yticklabels=['No Fraude', 'Fraude'],
            cbar_kws={'label': 'Cantidad'},
            annot_kws={'size': 16, 'weight': 'bold'})
plt.title('📊 Matriz de Confusión', fontsize=16, fontweight='bold')
plt.ylabel('Valor Real', fontsize=12)
plt.xlabel('Predicción', fontsize=12)
plt.tight_layout()
plt.show()

# Interpretación
tn, fp, fn, tp = cm.ravel()
print("\n🔍 Interpretación de la Matriz:")
print("="*50)
print(f"✅ Verdaderos Negativos (TN): {tn:>6,} - Correctamente identificados como NO fraude")
print(f"❌ Falsos Positivos (FP):     {fp:>6,} - Incorrectamente marcados como fraude")
print(f"❌ Falsos Negativos (FN):     {fn:>6,} - Fraudes no detectados")
print(f"✅ Verdaderos Positivos (TP): {tp:>6,} - Fraudes correctamente detectados")
print("="*50)

### 📊 6.2: Reporte de Clasificación

Métricas detalladas de rendimiento.

In [ ]:
# Reporte de clasificación
print("\n📊 REPORTE DE CLASIFICACIÓN")
print("="*70)
print(classification_report(y_test, y_pred, 
                          target_names=['No Fraude', 'Fraude'],
                          digits=4))

# Métricas adicionales
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n🎯 MÉTRICAS GLOBALES:")
print("="*70)
print(f"Accuracy (Exactitud):     {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision (Precisión):    {precision:.4f} ({precision*100:.2f}%)")
print(f"Recall (Sensibilidad):    {recall:.4f} ({recall*100:.2f}%)")
print(f"F1-Score:                 {f1:.4f}")
print(f"ROC-AUC Score:            {roc_auc:.4f}")
print("="*70)

print("\n💡 Interpretación:")
print(f"   ✅ De cada 100 predicciones de fraude, {precision*100:.1f} son correctas (Precision)")
print(f"   ✅ Detectamos el {recall*100:.1f}% de todos los fraudes reales (Recall)")
print(f"   ✅ Balance entre ambas: {f1:.4f} (F1-Score)")

### 📈 6.3: Curvas ROC y Precision-Recall

In [ ]:
# Calcular métricas para las curvas
fpr, tpr, thresholds_roc = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

precision_vals, recall_vals, thresholds_pr = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_vals, precision_vals)

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Curva ROC
axes[0].plot(fpr, tpr, color='#3498db', linewidth=2, 
            label=f'ROC Curve (AUC = {roc_auc:.4f})')
axes[0].plot([0, 1], [0, 1], color='gray', linestyle='--', linewidth=1, label='Random')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('📈 Curva ROC (Receiver Operating Characteristic)', 
                 fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Curva Precision-Recall
axes[1].plot(recall_vals, precision_vals, color='#e74c3c', linewidth=2,
            label=f'PR Curve (AUC = {pr_auc:.4f})')
axes[1].set_xlabel('Recall (Sensibilidad)', fontsize=12)
axes[1].set_ylabel('Precision (Precisión)', fontsize=12)
axes[1].set_title('📊 Curva Precision-Recall', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Resultados de las Curvas:")
print("="*60)
print(f"ROC-AUC:           {roc_auc:.4f} (Excelente si > 0.90)")
print(f"Precision-Recall:  {pr_auc:.4f} (Excelente si > 0.80)")
print("="*60)
print("\n💡 La curva PR es más informativa para datasets desbalanceados")

## 🎓 Conclusiones

### ✅ Lo que demostramos en este notebook:

1. **📊 Aprendizaje Iterativo:**
   - Visualizamos cómo el error disminuye con cada árbol
   - Cada modelo aprende de los errores del anterior

2. **⚖️ Regularización:**
   - Comparamos modelos con y sin regularización
   - Demostramos cómo L1 y L2 previenen el sobreajuste

3. **🔍 Selección de Variables:**
   - Mostramos las 3 métricas: Gain, Weight, Cover
   - Identificamos las variables más importantes

4. **🌳 Splits y Thresholds:**
   - Visualizamos cómo el modelo toma decisiones
   - Cada nodo representa una división basada en un umbral

5. **📈 Evaluación Completa:**
   - Matriz de confusión
   - Métricas apropiadas para datos desbalanceados
   - Curvas ROC y Precision-Recall

### 🎯 Resultados Finales:

XGBoost demostró ser altamente efectivo para detectar fraude:
- ✅ Alta precisión en la detección de fraudes
- ✅ Manejo efectivo del desbalanceo de clases
- ✅ Selección automática de variables importantes
- ✅ Prevención de sobreajuste mediante regularización

---

**💡 XGBoost es uno de los algoritmos más utilizados en la Ciencia de Datos moderna gracias a su:**
- 🚀 Alta precisión
- ⚡ Velocidad de entrenamiento
- 🛡️ Robustez ante sobreajuste
- 🔍 Capacidad de selección de variables

---

## 📚 Referencias

- Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System
- Documentación oficial de XGBoost: https://xgboost.readthedocs.io/
- Dataset: Credit Card Fraud Detection (Kaggle)

---

**Creado por:** Mariana Pérez Pérez, Yoriel Navier Carvajalino Vidal, Adriana Lucia Castro Carreño  
**Materia:** Ciencia de Datos  
**Fecha:** 2024